In [ ]:
import pandas as pd
from pathlib import Path
from PIL import Image
import torch
from torchvision import transforms
from ultralytics import YOLO
from sklearn.metrics import classification_report

: 

In [ ]:
# 모델 로드
model_path = "../../src/models/01_YOLO_cls_saved_model/weights/best.pt"
model = YOLO(model_path)

In [30]:
# 클래스 이름 설정
class_names = sorted([
    'female_10s', 'female_20s', 'female_30s', 'female_40s', 'female_50s', 'female_60s',
    'male_10s', 'male_20s', 'male_30s', 'male_40s', 'male_50s', 'male_60s'
])
class_to_idx = {cls_name: i for i, cls_name in enumerate(class_names)}

In [ ]:
# Validation CSV 로드
val_df = pd.read_csv("../../data/processed/01_processed_data/validation/val_labels.csv")
val_df["class_name"] = val_df["sex"] + "_" + val_df["age_class"]

# 이미지 경로 설정
val_image_dir = Path("../../data/processed/01_processed_data/validation/resized_faces")

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [33]:
# 예측
y_true, y_pred = [], []

for _, row in val_df.iterrows():
    img_path = val_image_dir / row["filename"]
    if not img_path.exists():
        continue

    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0)

    with torch.no_grad():
        results = model(img_tensor, verbose=False)
        pred = results[0].probs.top1  # 이건 int
        y_pred.append(pred)

    y_true.append(class_to_idx[row["class_name"]])

In [34]:
print(classification_report(y_true, y_pred, target_names=list(class_to_idx.keys())))

              precision    recall  f1-score   support

  female_10s       0.32      0.01      0.02       620
  female_20s       0.13      0.66      0.22       260
  female_30s       0.02      0.03      0.02       150
  female_40s       0.11      0.04      0.06       350
  female_50s       0.00      0.00      0.00       230
  female_60s       0.02      0.14      0.03        80
    male_10s       0.19      0.05      0.08       630
    male_20s       0.01      0.01      0.01       180
    male_30s       0.08      0.15      0.11       140
    male_40s       0.10      0.04      0.06       250
    male_50s       0.08      0.05      0.06       200
    male_60s       0.03      0.04      0.03       110

    accuracy                           0.09      3200
   macro avg       0.09      0.10      0.06      3200
weighted avg       0.14      0.09      0.06      3200



# ⭐문제점 발견 및 보완사항
너무 낮게 나와서 찾아보니  

1. 성별 + 연령대인 Multi-label 분류에는 YOLO보다는 CNN 전이학습 모델이 훨씬 더 적합하다고 한다.  

2. YOLOv8-cls는 단일 클래스만 예측하도록 설계되어 있다.  
그래서 'female_20s' 과 같은 복합 클래스를 단일 클래스로 봐야 한다.  
그런데 클래스가 12개가 되니까 학습 분산이 생기고, 이로 인해 하나하나 정확도가 떨어지는 문제가 생긴다.  

3. 데이터 증강이 필요하다.  
밝기, 줌 사이즈, 각도 등에 대한 증강을 통해 머리카락, 안경, 주름, 조명 같은 성별과 연령 구분에 도움을 주어야 한다.  